In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
# Data Loading.
train_df = pd.read_csv('../input/bike-sharing-demand/train.csv')
test_df = pd.read_csv('../input/bike-sharing-demand/test.csv')

# Data Exploring.

In [ ]:
print(train_df.shape)
print(test_df.shape)

In [ ]:
train_df.head(10)

In [ ]:
train_df.info()

In [ ]:
test_df.head(10)

In [ ]:
test_df.info()

In [ ]:
# datatime 컬럼에서 년,월,일,시간,요일 값 분리저장
train_df['datetime'] = pd.to_datetime(train_df['datetime'])
test_df['datetime'] = pd.to_datetime(test_df['datetime'])

train_df['year'] = train_df['datetime'].apply(lambda x: x.year)
train_df['month'] = train_df['datetime'].apply(lambda x: x.month)
train_df['day'] = train_df['datetime'].apply(lambda x: x.day)
train_df['hour'] = train_df['datetime'].apply(lambda x: x.hour)

test_df['year'] = test_df['datetime'].apply(lambda x: x.year)
test_df['month'] = test_df['datetime'].apply(lambda x: x.month)
test_df['day'] = test_df['datetime'].apply(lambda x: x.day)
test_df['hour'] = test_df['datetime'].apply(lambda x: x.hour)

In [ ]:
train_df.head(5)

In [ ]:
test_df.head(5)

In [ ]:
# year, month, day, hour 추가로 datetime 삭제.
# casual, registerd 컬럼도 test data에는 없기때문에 삭제처리.
train_df = train_df.drop(['datetime', 'casual', 'registered'], axis=1)
test_df = test_df.drop(['datetime'], axis=1)

In [ ]:
train_df.head(5)

In [ ]:
test_df.head(5)

# Find out the characteristics of each feature through the graph

In [ ]:
#년도별, 월별, 일자별, 시간대별 대여횟수 파악.
# 년도별 Count
fig = plt.figure(figsize=[12,10])
ax1 = fig.add_subplot(2,2,1)
ax1 = sns.barplot(x='year',y='count',data=train_df.groupby('year')['count'].mean().reset_index())

# 월별 Count
ax2 = fig.add_subplot(2,2,2)
ax2 = sns.barplot(x='month',y='count',data=train_df.groupby('month')['count'].mean().reset_index())

# 일자별 Count
ax3 = fig.add_subplot(2,2,3)
ax3 = sns.barplot(x='day',y='count',data=train_df.groupby('day')['count'].mean().reset_index())

# 시간별 Count
ax4 = fig.add_subplot(2,2,4)
ax4 = sns.barplot(x='hour',y='count',data=train_df.groupby('hour')['count'].mean().reset_index())

* 연도별 Count를 통해 2011년에 비해 2012년도에 대여량이 증가함.
  시간이 지날수록 홍보효과로 사용인원이 늘어난 것으로 추정할수 있음.
* 월별 Count를 통해 6월에 대여 횟수가 가장 높고 1월에 가장 낮음.
* 일별 Count를 보면 편차가 크지 않는것으로 보임.
* 시간대별 Count를 살펴보면 아침 출근시간, 저녁 퇴근시간에 가장 대여량이 많은것으로 확인됨.

In [ ]:
#계절, 휴일, 작업일, 날씨별 대여횟수 파악
fig = plt.figure(figsize=[12,10])

# WrokingDay별 Count
ax1 = fig.add_subplot(2,2,1)
ax1 = sns.barplot(x='workingday',y='count',data=train_df.groupby('workingday')['count'].mean().reset_index())

# Holiday별 Count
ax2 = fig.add_subplot(2,2,2)
ax2 = sns.barplot(x='holiday',y='count',data=train_df.groupby('holiday')['count'].mean().reset_index())

# Season별 Count
ax3 = fig.add_subplot(2,2,3)
ax3 = sns.barplot(x='season',y='count',data=train_df.groupby('season')['count'].mean().reset_index())

# Weather별 Count
ax4 = fig.add_subplot(2,2,4)
ax4 = sns.barplot(x='weather',y='count',data=train_df.groupby('weather')['count'].mean().reset_index())

* 주말(휴일) 이냐 근무일 이냐에 따라서는 대여 횟수 차이가 크게 나지않은것으로 보여짐.
* 계절별로는 가을에 가장 대여횟수가 높고 봄에 겨울,봄에 가장낮은 것으로 확인됨.
* 날씨는 맑은날 가장 대여횟수가 높고 날씨는 안좋을수록 낮은것으로 확인됨.

# Data Cleansing.

In [ ]:
train_df.head(10)

In [ ]:
# windspeed 값에 연속된 0값이 있어 Value별 Count 확인
train_df['windspeed'].value_counts()

In [ ]:
# 풍속값 그래프로 확인결과 Train, Test 모두 0값이 많음.
fig, axes = plt.subplots(nrows=2)
fig.set_size_inches(15,15)

plt.sca(axes[0])
plt.xticks(rotation=30, ha='right')
axes[0].set(ylabel='Count',title="< Train Data Windspeed >")
sns.countplot(data=train_df, x="windspeed", ax=axes[0])

plt.sca(axes[1])
plt.xticks(rotation=30, ha='right')
axes[1].set(ylabel='Count',title="< Test Data Windspeed >")
sns.countplot(data=test_df, x="windspeed", ax=axes[1])

In [ ]:
# 풍속이 0인 Row에 간단하게 평균값을 구해서 넣어줌.
train_df.loc[train_df["windspeed"] == 0, "windspeed"] = train_df["windspeed"].mean()
test_df.loc[test_df["windspeed"] == 0, "windspeed"] = test_df["windspeed"].mean()

In [ ]:
# 풍속값 교정후 그래프 확인
fig, axes = plt.subplots(nrows=2)
fig.set_size_inches(15,15)

plt.sca(axes[0])
plt.xticks(rotation=30, ha='right')
axes[0].set(ylabel='Count',title="< Train Data Windspeed >")
sns.countplot(data=train_df, x="windspeed", ax=axes[0])

plt.sca(axes[1])
plt.xticks(rotation=30, ha='right')
axes[1].set(ylabel='Count',title="< Test Data Windspeed >")
sns.countplot(data=test_df, x="windspeed", ax=axes[1])

# Check the distribution of target(Count) values</br>
  * Target 값의 분포는 정규 분포 형태가 가장 좋음.</br> 그렇지 않고 왜곡된 경우 회귀 예측 성능이 저하되는 경우가 발생.

In [ ]:
sns.distplot(train_df['count'])

* count 값이 정규 분포 형태가 아니어서 로그값을 적용하여 변환처리(정규분포형태)

In [ ]:
sns.distplot(np.log1p(train_df['count']))

* 정규분포 형태는 아니지만 변환하기 전보다 왜곡정도가 많이 향상됨.

In [ ]:
# 변경전 count값
train_df['count']

In [ ]:
# 로그 적용후 Count값 (*****)
train_df['count'] = np.log1p(train_df['count'])
train_df['count']

# Model Selection

* 선형 회귀 모델( Linear Regression Model )

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso

In [ ]:
# 캐글의 성능 평가지표는 RMSLE(Root Mean Square Log Error)
# 사이킷런은 RMSLE를 제공하지 않아 Function Define.
def rmsle(y, pred):
    log_y = np.log1p(y)
    log_pred = np.log1p(pred)
    squared_error = (log_y - log_pred)**2   #MSLE
    rmsle = np.sqrt(np.mean(squared_error)) #RMSLE
    return rmsle

In [ ]:
# Train,Test Data Split
X_train, X_test, y_train, y_test = train_test_split(train_df.drop(['count'], axis=1), train_df['count'], test_size=0.3)

In [ ]:
lr_reg = LinearRegression()

#학습
lr_reg.fit(X_train, y_train)

#예측
pred = lr_reg.predict(X_test)

# log1p(target) -> expm1(target) ( ***** )
y_test_exp = np.expm1(y_test)
pred_exp = np.expm1(pred)

#평가
print('RMSLE:', rmsle(y_test_exp, pred_exp))

* 선형회귀 이기 때문에 평가지표 값이 낮을수록 좋은 점수임.</br>
  평가점수가 좋지 않은것으로 판단됨.

In [ ]:
# 각피쳐의 회귀 계수값 시각화
coef = pd.Series(lr_reg.coef_, index=X_train.columns)
coef_sort = coef.sort_values(ascending=False)
sns.barplot(x=coef_sort.values, y=coef_sort.index)

* 카테고리형 Feature 들이 숫자형 값으로 되어있어 회귀계수 값이 크게 나타나고 있는것으로 나타남.(실제 대여횟수에 크게 영향을 미치지 않음.)</br>
  Feature값중 숫자형 카테고리 값을 선형회귀에 사용할 경우 회귀계수를 연산할때에 숫자형 값에 크게 영향을 받을수 있어 원-핫 인코딩을 적용해 변환
  
  

In [ ]:
#train_df = pd.get_dummies(train_df, columns=['year', 'month', 'day', 'hour', 'holiday', 'workingday', 'season', 'weather'])
#test_df = pd.get_dummies(test_df, columns=['year', 'month', 'day', 'hour', 'holiday', 'workingday', 'season', 'weather'])

# train, test data 모두 원-핫 인코딩( One-Hot Encoding ) 적용. ( day 값은 Train Data와 Test Data 일수가 틀려 빼버림. 회귀계수값도 낮은편임)
train_df = pd.get_dummies(train_df, columns=['year', 'month', 'hour', 'holiday', 'workingday', 'season', 'weather'])
test_df = pd.get_dummies(test_df, columns=['year', 'month', 'hour', 'holiday', 'workingday', 'season', 'weather'])

In [ ]:
print(train_df.shape)
print(test_df.shape)

In [ ]:
train_df.info()

In [ ]:
test_df.info()

In [ ]:
# train_df와 test_df의 shape를 맞춰주기 위해 align 적용. train data 와 test data의 Day(일) 값이 틀림.
#train_df, test_df = train_df.align(test_df, join='left', axis=1)
#test_df = test_df.drop(['count'], axis=1)
#print(train_df.shape)
#print(test_df.shape)
#train_df.info()
#test_df.info()

In [ ]:
# 훈련 데이터와 테스트 테이터 다시 분리( After One-Hot Encoding )
X_train, X_test, y_train, y_test = train_test_split(train_df.drop(['count'], axis=1), train_df['count'], test_size=0.3)

In [ ]:
# 선형회귀 모델 학습/예측/평가하기
lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train)
pred = lr_reg.predict(X_test)

y_test_exp = np.expm1(y_test)
pred_exp = np.expm1(pred)
print('LinearRegression RMSLE:', rmsle(y_test_exp, pred_exp))

In [ ]:
# Ridge 모델 학습/예측/평가
ridge_reg = Ridge(alpha=10)
ridge_reg.fit(X_train, y_train)
pred = ridge_reg.predict(X_test)

y_test_exp = np.expm1(y_test)
pred_exp = np.expm1(pred)
print('Ridge RMSLE:', rmsle(y_test_exp, pred_exp))

In [ ]:
# Lasso 모델 학습/예측/평가
lasso_reg = Lasso(alpha=0.01)
lasso_reg.fit(X_train, y_train)
pred = lasso_reg.predict(X_test)

y_test_exp = np.expm1(y_test)
pred_exp = np.expm1(pred)
print('Lasso RMSLE:', rmsle(y_test_exp, pred_exp))

In [ ]:
# 선형회귀 회귀계수 상위25개 피쳐 추출.
coef = pd.Series(lr_reg.coef_, index=X_train.columns)
coef_sort = coef.sort_values(ascending=False)[:25]
sns.barplot(x=coef_sort.values, y=coef_sort.index)

In [ ]:
# Ridge 회귀계수 상위25개 피쳐 추출.
coef = pd.Series(ridge_reg.coef_, index=X_train.columns)
coef_sort = coef.sort_values(ascending=False)[:25]
sns.barplot(x=coef_sort.values, y=coef_sort.index)

In [ ]:
# Lasso 회귀계수 상위25개 피쳐 추출.
coef = pd.Series(lasso_reg.coef_, index=X_train.columns)
coef_sort = coef.sort_values(ascending=False)[:25]
sns.barplot(x=coef_sort.values, y=coef_sort.index)

* 원핫 인코딩 적용후 날씨, 계절, 월, 주말/주중, 휴일등 대여횟수와 관계가 있는 피쳐의 회귀계수값이 높아짐.

## Tree-based Regression.

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [ ]:
rf_reg = RandomForestRegressor(n_estimators=50)
rf_reg.fit(X_train, y_train)
pred = rf_reg.predict(X_test)

y_test_exp = np.expm1(y_test)
pred_exp = np.expm1(pred)
print('RandomForestRegressor RMSLE:', rmsle(y_test_exp, pred_exp))

# Hyper-parameter Tuning.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_log_error,mean_squared_error, r2_score,mean_absolute_error # for regression

# GridSearchCv를 이용한 하이퍼 파라미터.
tuned_parameters = {'n_estimators': [100,200,300,500], 'max_depth': [None, 1, 2, 3], 'min_samples_split': [1, 2, 3]}

rf_reg = GridSearchCV(RandomForestRegressor(), tuned_parameters, cv=5, scoring='neg_mean_squared_log_error', n_jobs=-1, verbose=1)
rf_reg.fit(X_train, y_train)
pred = rf_reg.predict(X_test)

y_test_exp = np.expm1(y_test)
pred_exp = np.expm1(pred)
print('RandomForestRegressor RMSLE:', rmsle(y_test_exp, pred_exp))

rf_reg.best_params_


# 예측(Prediction), 제출(Submission)

In [ ]:
X_train = train_df.drop(['count'], axis=1)
y_train = train_df['count']

# Test Data ( test.csv )
X_test = test_df

In [ ]:
X_test.head(10)

* 가장 성능이 평가지표가 높은 RandomForestRegressor 모델로 하이퍼파라미터 적용후 Test Data 예측값 제출처리.

In [ ]:
# GridSearchCV의 refit으로 이미 학습이 된 estimator 반환
best_rf_reg = rf_reg.best_estimator_

# GridSearchCv로 최적의 파라미터 적용된 모델로 예측처리.
pred = best_rf_reg.predict(X_test)

pred_exp = np.expm1(pred)

print(pred[0:10])

In [ ]:
#실제값 Target값과 예측 Target값 그래프로 비교해보기.
y_train_exp = np.expm1(y_train)

fig,(ax1,ax2)= plt.subplots(ncols=2)
fig.set_size_inches(12,5)
sns.distplot(y_train_exp,ax=ax1,bins=50)
sns.distplot(pred_exp,ax=ax2,bins=50)

In [ ]:
# Submission Sample Data 확인.
submission = pd.read_csv('../input/bike-sharing-demand/sampleSubmission.csv')
submission

* 최초 모델로 예측한 값을 submission 파일로 만든후 제출하기.
* 튜닝후 제출파일을 submission_after_tunning 으로 변경.

In [ ]:
# Count값에 예측값 넣어주기
submission.loc[:, 'count'] = pred_exp
submission

In [ ]:
# 제출 파일명은 아무것이나 사용해도 무방.
submission.to_csv('submission_after_tunning.csv', index=False)